# Data Science Systems: Creating a segment of an ETL pipeline that will ingest and process raw data
## - Implementing data science systems rooted in SQL and other data sources like CSVs, Open Data, and other relational data sources, as well as APIs and data transformation
## Stephanie Fissel, October 20, 2022

## Import Packages

In [9]:
import json
import csv
import requests
import pandas as pd
import sqlite3

## Fetch remote data file by URL

### Import urllib library

In [3]:
from urllib.request import urlopen

### Store URL in url as parameter for urlopen

In [4]:
url = "https://holidays.abstractapi.com/v1/?api_key=574b7d06a79c4794a2e390123a19b857&country=US&year=2020"

### Store the response of URL

In [6]:
response = urlopen(url)

### Store the JSON response from url in data

In [10]:
data_json = json.loads(response.read())

### Print the json response

In [13]:
print(data_json)

[{'name': "New Year's Day", 'name_local': '', 'language': '', 'description': '', 'country': 'US', 'location': 'United States', 'type': 'National', 'date': '01/01/2020', 'date_year': '2020', 'date_month': '01', 'date_day': '01', 'week_day': 'Wednesday'}, {'name': 'World Braille Day', 'name_local': '', 'language': '', 'description': '', 'country': 'US', 'location': 'United States', 'type': 'Worldwide', 'date': '01/04/2020', 'date_year': '2020', 'date_month': '01', 'date_day': '04', 'week_day': 'Saturday'}, {'name': 'Epiphany', 'name_local': '', 'language': '', 'description': '', 'country': 'US', 'location': 'United States', 'type': 'Christian', 'date': '01/06/2020', 'date_year': '2020', 'date_month': '01', 'date_day': '06', 'week_day': 'Monday'}, {'name': "International Programmers' Day", 'name_local': '', 'language': '', 'description': '', 'country': 'US', 'location': 'United States', 'type': 'Worldwide', 'date': '01/07/2020', 'date_year': '2020', 'date_month': '01', 'date_day': '07', '

## Convert from JSON to SQLite database (with reduced number of columns from source to destination)

In [14]:
api_url = requests.get('https://holidays.abstractapi.com/v1/?api_key=574b7d06a79c4794a2e390123a19b857&country=US&year=2020')
data_json = api_url.json()

connection = sqlite3.connect('holiday.sqlite')
cursor = connection.cursor()
cursor.execute('Create Table if not exists holiday (name TEXT, date TEXT, type TEXT, country TEXT)')

columns = ['name','date','type', 'country']
for row in data_json:
    keys= tuple(row[c] for c in columns)
    cursor.execute('insert into holiday values(?,?,?,?)',keys)
    print(f'{row["name"]} data inserted Succefully')

connection.commit()
connection.close()

New Year's Day data inserted Succefully
World Braille Day data inserted Succefully
Epiphany data inserted Succefully
International Programmers' Day data inserted Succefully
Orthodox Christmas Day data inserted Succefully
Stephen Foster Memorial Day data inserted Succefully
Orthodox New Year data inserted Succefully
Lee-Jackson Day data inserted Succefully
Confederate Heroes' Day data inserted Succefully
World Religion Day data inserted Succefully
Robert E. Lee's Birthday data inserted Succefully
Martin Luther King Jr. Day data inserted Succefully
Civil Rights Day data inserted Succefully
Robert E. Lee's Birthday data inserted Succefully
Civil Rights Day data inserted Succefully
Robert E. Lee's Birthday data inserted Succefully
Robert E. Lee's Birthday data inserted Succefully
Idaho Human Rights Day data inserted Succefully
Chinese New Year data inserted Succefully
World Leprosy Day data inserted Succefully
International Customs Day data inserted Succefully
International Day of Commemor

## Produce informative error if file doesn't exist to save

In [15]:
try:
    with open('holiday.sqlite') as file:
        print("File present")
except FileNotFoundError:
    print('File is not present')

File present


## Convert from JSON to CSV (local file) written to disk

In [16]:
data = pd.read_json('https://holidays.abstractapi.com/v1/?api_key=574b7d06a79c4794a2e390123a19b857&country=US&year=2020')
data.to_csv('holiday.csv', index=None)

## Modify number of columns from source to destination: CSV file

### Remove columns: date_day, date_month, date_year, description, language, location, name_local

In [8]:
data.drop(['date', 'description', 'language', 'location', 'name_local'], inplace=True, axis=1)

### Add date column that combines weekday, date day, month, and year of holiday

In [9]:
data["date"] = (data["week_day"].astype(str) + " " + data["date_month"].astype(str) + "/" + data["date_day"].astype(str) + "/" + data["date_year"].astype(str))
data.drop(['date_day', 'date_month', 'date_year', 'week_day'], inplace=True, axis=1)

### Reorder columns

In [10]:
data = data[['name', 'date', 'type', 'country']]

### Update new CSV file written to disk (local file)

In [11]:
data.to_csv('holiday.csv', index=None)

## Produce informative error if file doesn't exist to update

In [12]:
try:
    with open('holiday.csv') as file:
        read_data = file.read()
except FileNotFoundError as fnf_error:
    print(fnf_error)

## Brief summary of data file ingestion for SQLite database

### Connecting to sqlite
#### Connection object

In [19]:
connection_obj = sqlite3.connect('holiday.sqlite')

#### Cursor object

In [20]:
cursor_obj = connection_obj.cursor()

### How many records and columns are in the SQLite database:

In [13]:
print("Number of Records: ")
cursor_obj.execute("SELECT * FROM HOLIDAY")
print(len(cursor_obj.fetchall()))

print("Number of Columns: ")
cursor_obj.execute("SELECT * FROM pragma_table_info('HOLIDAY')")
print(len(cursor_obj.fetchall()))

Number of Records: 
848
Number of Columns: 
4


### Close the connection

In [21]:
connection_obj.close()

## Brief summary of data file ingestion for CSV file

### How many records and columns are in the CSV file:

In [14]:
total_records = len(data.axes[0])
total_columns = len(data.axes[1])
print("Number of Records: "+ str(total_records))
print("Number of Columns: "+ str(total_columns))

Number of Records: 424
Number of Columns: 4
